In [ ]:
!pip install sentence-transformers

In [ ]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.6 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade pypdf2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.4 MB/s eta 0:00:00


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 65.8 MB/s eta 0:00:00


In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 41.9 MB/s eta 0:00:00


In [ ]:
import os

# Path to the FAISS index file
faiss_index_path = "faiss_index_file.index"

# Check if the file exists, then delete it
if os.path.exists(faiss_index_path):
    os.remove(faiss_index_path)
    print(f"Deleted existing FAISS index at: {faiss_index_path}")
else:
    print("No existing FAISS index found.")

No existing FAISS index found.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from wordcloud import WordCloud
from IPython.display import IFrame
from sentence_transformers import SentenceTransformer
import faiss
from PyPDF2 import PdfReader
from PyPDF2 import PdfReader
import os

In [ ]:
import fitz  # PyMuPDF
from datasets import Dataset

def chunk_text(text, chunk_size=100, overlap=25):
    """
    Split the text into chunks of a specified size with overlap.
    """
    words = text.split()
    chunks = []
    for i in range(0, len(words), chunk_size - overlap):
        chunk = " ".join(words[i:i + chunk_size])
        if chunk:  # Ensure no empty chunks
            chunks.append(chunk)
    return chunks

def extract_text_with_chunks(pdf_path, output_txt_path, chunk_size=100, overlap=25):
    """
    Extract text with page numbers, create chunks, and save to a dataset.
    """
    try:
        # Open the PDF
        pdf_document = fitz.open(pdf_path)

        # Prepare a list for dataset entries
        dataset_entries = []

        # Open the output file for writing
        with open(output_txt_path, "w", encoding="utf-8") as output_file:
            # Iterate through each page in the PDF
            for page_number in range(len(pdf_document)):
                # Get the page object
                page = pdf_document[page_number]

                # Extract text from the page
                text = page.get_text().strip()

                # Write the full-page text to the output file
                if text:  # Only process non-empty pages
                    formatted_text = f"Page: {page_number + 1} - Text: “{text}”\n\n"
                    output_file.write(formatted_text)

                    # Create overlapping chunks for the page text
                    chunks = chunk_text(text, chunk_size=chunk_size, overlap=overlap)

                    # Add each chunk to the dataset entries with page metadata
                    for chunk in chunks:
                        dataset_entries.append({
                            "Page": page_number + 1,
                            "Chunk": chunk,
                            "Text": text
                        })

        print(f"Formatted text with chunks saved to {output_txt_path}")

        # Create a dataset from the entries
        dataset = Dataset.from_list(dataset_entries)

        # Display dataset info
        print("Dataset created successfully!")
        print(dataset)

        return dataset
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Input PDF path and output text file path
input_pdf_path = "selected_pages.pdf"
output_text_file = "extracted_text_with_chunks.txt"

# Call the function to extract text, chunk it, and create a dataset
dataset = extract_text_with_chunks(input_pdf_path, output_text_file)

# Optionally, print the first few rows of the dataset
if dataset:
    print(dataset[:5])

Formatted text with chunks saved to extracted_text_with_chunks.txt
Dataset created successfully!
Dataset({
    features: ['Page', 'Chunk', 'Text'],
    num_rows: 4891
})
{'Page': [1, 1, 1, 1, 1], 'Chunk': ['Preface Welcome to Psychology 2e, an OpenStax resource. This textbook was written to increase student access to high-quality learning materials, maintaining highest standards of academic rigor at little to no cost. About OpenStax OpenStax is a nonprofit based at Rice University, and it’s our mission to improve student access to education. Our first openly licensed college textbook was published in 2012, and our library has since scaled to over 35 books for college and AP® courses used by hundreds of thousands of students. OpenStax Tutor, our low-cost personalized learning tool, is being piloted in college courses throughout the country. Through', 'courses used by hundreds of thousands of students. OpenStax Tutor, our low-cost personalized learning tool, is being piloted in college c

In [ ]:
# Optional: Show the first few rows
print(dataset[:1])

{'Page': [1], 'Chunk': ['Preface Welcome to Psychology 2e, an OpenStax resource. This textbook was written to increase student access to high-quality learning materials, maintaining highest standards of academic rigor at little to no cost. About OpenStax OpenStax is a nonprofit based at Rice University, and it’s our mission to improve student access to education. Our first openly licensed college textbook was published in 2012, and our library has since scaled to over 35 books for college and AP® courses used by hundreds of thousands of students. OpenStax Tutor, our low-cost personalized learning tool, is being piloted in college courses throughout the country. Through'], 'Text': ['Preface\nWelcome to Psychology 2e, an OpenStax resource. This textbook was written to increase student access to\nhigh-quality learning materials, maintaining highest standards of academic rigor at little to no cost.\nAbout OpenStax\nOpenStax is a nonprofit based at Rice University, and it’s our mission to i

In [ ]:
# Replace 'your_file.pdf' with the path to your PDF
# IFrame('/content/selected_pages.pdf', width=800, height=600)

In [ ]:
# Replace 'your_file.json' with the path to your JSON file
df = pd.read_json('queries.json')
# Display the DataFrame
df

,query_id,question
0,1,What is the scientific method in psychology?
1,2,What are the basic parts of a neuron?
2,3,What are the stages of sleep?
3,4,What is operant conditioning?
4,5,What is problem-solving in psychology?
5,6,What are the three stages of memory?
6,7,What are the key components of emotion?
7,8,What are the major personality traits in the F...
8,9,What is social psychology?
9,10,What is the sociocultural model in therapy?


In [ ]:
import torch

# Check if GPU is available and set device accordingly
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


In [ ]:
# Initialize the embedding model
embedding_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2',device=device)

# Generate embeddings
embeddings = embedding_model.encode([item["Chunk"] for item in dataset], convert_to_numpy=True,show_progress_bar=True)

# Add embeddings to the dataset (optional)
dataset = dataset.add_column("Embeddings", embeddings.tolist())

# Display the dataset with embeddings
print(dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/153 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
book_data_with_overlap =[
    {
        "page": row["Page"],
        "chunk": row["Chunk"],
        "text": row["Text"],
        "embeddings": row["Embeddings"]
    }
    for row in dataset
]

book_data_with_overlap[:1]

In [ ]:
# Step 5: Create a FAISS index (flat index for simplicity)
# Create a FAISS index
dimension = embeddings.shape[1]  # Embedding dimension
index = faiss.IndexFlatL2(dimension)  # L2 (Euclidean) distance
index.add(embeddings)  # Add embeddings to the index

print(f"Number of embeddings in the index: {index.ntotal}")

In [ ]:
# Assuming you have a FAISS index (e.g., 'index' variable)
faiss.write_index(index, 'faiss_index_file.index')

In [ ]:
!pip install huggingface_hub

In [ ]:
# FAISS search function to return all matching texts based on the similarity threshold
def search_faiss(query, index, texts, threshold=0.5):
    """
    Search the FAISS index for all texts with similarity above a given threshold.
    Args:
        query (str): The query text.
        index (faiss.Index): The FAISS index.
        texts (list): The list of texts corresponding to the index embeddings.
        threshold (float): The distance threshold to filter matches (lower is more similar).

    Returns:
        distances (list): List of distances for all matching texts.
        matching_texts (list): List of all matching texts based on the threshold.
        indices (list): List of indices for matching texts.
    """
    # Generate the embedding for the query
    query_embedding = embedding_model.encode([query]).astype(np.float32)

    # Search FAISS for all potential matches
    distances, indices = index.search(query_embedding, len(texts))  # Search for all texts

    # Filter based on the threshold
    matching_indices = [i for i, dist in zip(indices[0], distances[0]) if dist <= threshold]
    matching_distances = [dist for dist in distances[0] if dist <= threshold]
    matching_texts = [texts[i] for i in matching_indices]

    return matching_distances, matching_texts, matching_indices


llm_tokenizer = AutoTokenizer.from_pretrained("gpt2")
llm_model = AutoModelForCausalLM.from_pretrained("gpt2")

# Set the pad_token to be the eos_token (End of Sequence token)
llm_tokenizer.pad_token = llm_tokenizer.eos_token

def generate_answer_with_llm(query, context):
    # Prepare the input prompt for the LLM
    prompt = f"{context}"

    # Tokenize the prompt and ensure proper padding/truncation
    inputs = llm_tokenizer(prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

    # Generate the response with `max_new_tokens` to control output length
    outputs = llm_model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],  # Include the attention mask
        max_new_tokens=400,                     # Maximum number of tokens to generate
        temperature=0.5,                        # Sampling temperature
        top_p=0.9,                              # Nucleus sampling for diversity
        top_k=50,                               # Limit top-k tokens for sampling
        num_beams=5,                            # Beam search for more coherent responses
        no_repeat_ngram_size=2,                 # Avoid repetitive text
        pad_token_id=llm_tokenizer.pad_token_id, # Handle padding correctly
        do_sample=True                          # Enable sampling to use `temperature` and `top_p`
    )

    # Decode the output
    answer = llm_tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Perform any post-processing if necessary (e.g., clean unwanted parts of text)
    answer = answer.replace("Answer:", "").strip()
    return answer

In [ ]:
def search_faiss(query, index, texts,k=2):
    """
    Search the FAISS index for all texts with similarity above a given threshold.

    Args:
        query (str): The query text.
        index (faiss.Index): The FAISS index.
        texts (list): The list of texts (dictionaries) with "text", "page", and "embedding".
        threshold (float): The distance threshold to filter matches (lower is more similar).

    Returns:
        list: A list of results containing matching distances, texts, and page numbers.
    """
    # Generate the embedding for the query
    query_embedding = embedding_model.encode([query]).astype(np.float32)

    # Search FAISS for all potential matches
    distances, indices = index.search(query_embedding, len(texts))  # Search for all texts

    # Filter based on the threshold
    results = []
    for idx, dist in zip(indices[0], distances[0]):
        if dist <= threshold:  # Check the threshold condition
            results.append({
                "distance": dist,
                "chunk": texts[idx]["chunk"],
                "text": texts[idx]["text"],  # Extract the matching text
                "page": texts[idx]["page"],   # Extract the corresponding page number
            })

    return results

In [ ]:
# List of questions to search
questions = df["question"].to_list()
# Load the FAISS index (or create it if you haven't saved it)
index = faiss.read_index('faiss_index_file.index')

In [ ]:
import torch

# Check for GPU availability
if torch.cuda.is_available():
    print(f"GPU available: {torch.cuda.get_device_name(0)}")
else:
    print("No GPU available. Using CPU.")

In [ ]:
# Initialize an empty list to store results with pages and texts
final_results = []

# Define the distance threshold
threshold = 0.25

for question in questions:
    # Generate the embedding for the query
    query_embedding = embedding_model.encode([question]).astype(np.float32)

    # Perform the search in FAISS index for all possible results
    # Assuming the FAISS index has enough data points
    distances, indices = index.search(query_embedding, len(book_data_with_overlap))  # Search all data points

    # Initialize lists to store matching results
    results = []

    # Iterate through all search results and filter by threshold
    for idx, dist in zip(indices[0], distances[0]):
        if dist > threshold and idx < len(book_data_with_overlap):  # Check threshold and ensure index is valid
            result = book_data_with_overlap[idx]  # Get the corresponding result
            result["distance"] = dist  # Add the distance to the result
            results.append(result)

    # Combine all matching texts as context for the LLM
    context = " ".join([result['chunk'] for result in results])  # Combine the text content for context

    # Generate an answer with the LLM using the context from the matching texts
    answer = generate_answer_with_llm(question, context)  # Assume this function generates the answer

    # Store the relevant details in final_results
    for result in results:
        result_page = result['page']
        result_chunk = result['chunk']
        final_results.append({
            "question": question,
            "result_page": result_page,
            "result_text": result_chunk,
            "distance": result['distance'],
            "generated_answer": answer
        })

    # Step 3: Display the results for the current question
    print(f"Question: {question}")
    print(f"Generated Answer: {answer}")
    print("=" * 50)  # Separator for clarity

# final_results now contains the page, text, distance, and generated answer for all results above threshold.

In [ ]:
import csv
import json

def json_to_csv(json_data, csv_filename):
    # Open the CSV file in write mode
    with open(csv_filename, mode='w', newline='', encoding='utf-8') as csv_file:
        # Define the fieldnames (column headers)
        fieldnames = ['question', 'result_page', 'result_text', 'distance', 'generated_answer']

        # Create a CSV DictWriter object
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

        # Write the header to the CSV
        writer.writeheader()

        # Write each row of the JSON data to the CSV
        for entry in json_data:
            writer.writerow(entry)

# Example usage:
json_data = final_results  # Assuming final_results is your list of JSON data
csv_filename = 'results.csv'  # Name of the output CSV file
json_to_csv(json_data, csv_filename)
pd.read_csv("results.csv")

In [ ]:
content_df=pd.read_csv("content_df.csv")
content_df

In [ ]:
# Words to filter
words_to_filter = ["CONTENTS","CHAPTER"]

# Filter rows where 'text' column contains any of the words
df_filtered = content_df[~content_df['topic'].str.contains('|'.join(words_to_filter), case=False, na=False)]

# Output the result
df_filtered

In [ ]:
# Identify rows where the 'topic' contains 'CHAPTER'
chapter_rows = content_df[content_df['topic'].str.contains('CHAPTER', case=False, na=False)]

# Get the index of the chapter rows
chapter_indices = chapter_rows.index

# Extract the rows with 'Chapter' and the row below them
extracted_rows = pd.concat([content_df.loc[chapter_indices], content_df.loc[chapter_indices + 1]])

# Ensure we don't exceed the DataFrame index range (handle last row case)
extracted_rows = extracted_rows[extracted_rows.index < len(content_df)]

# Reset index for the final output
extracted_rows = extracted_rows.reset_index(drop=True)

# Function to drop rows that contain "Chapter" in any column
def drop_rows_with_chapter(df):
    # Apply the lambda function to each cell and check if 'Chapter' is present
    df = df[~df['topic'].str.contains('CHAPTER', case=False, na=False)]
    return df

# Apply the function to drop rows containing "Chapter"
df_cleaned = drop_rows_with_chapter(extracted_rows)

# Output the final cleaned DataFrame
df_cleaned

In [ ]:
df_cleaned.rename(columns={"topic": "chapter","page_no":"chapter_page"}, inplace=True)
df_cleaned

In [ ]:
# Perform the merge based on the cross join
merged_df = df_cleaned.merge(df_filtered, how='cross')

# Filter rows where 'C' is less than 'A'
filtered_rows = merged_df[merged_df['chapter_page'] <= merged_df['page_no']]

# Find the nearest and lower value for each 'A' in df1
nearest_rows = (
    filtered_rows.loc[filtered_rows.groupby('page_no')['chapter_page'].idxmax()]
)  # idxmax ensures we get the largest 'C' < 'A'

# Reset the index for a clean result
nearest_rows.reset_index(drop=True, inplace=True)

# Output the result
nearest_rows

In [ ]:
values_to_check_2 = results_["result_page"].to_list()
df3 = nearest_rows[["page_no"]]

# Sort data2 values (if not already sorted)
df3 = df3.sort_values('page_no').reset_index(drop=True)

# Function to check if a value lies between two consecutive rows in data2
def find_matching_values(value, ranges):
    for i in range(len(ranges) - 1):
        if ranges.iloc[i]['page_no'] <= value <= ranges.iloc[i + 1]['page_no']:
            return ranges.iloc[i]['page_no']  # Return the start of the matching range
    return None  # Return None if no match is found

# Apply the function to the list of values to check and keep None values
matching_values_1 = [find_matching_values(val, df3) for val in values_to_check_2]

# Check the length of matching_values_1
print(f"Length of matching_values_1: {len(matching_values_1)}")

# Output the result
print("Matching Values:", matching_values_1)

In [ ]:
results_["page_no"]=matching_values_1
results_

In [ ]:
results_[~results_["page_no"].notna()]

In [ ]:
results_.isna().sum()

In [ ]:
final_df=results_.merge(nearest_rows, on="page_no", how="left")
final_df

In [ ]:
final_df["sections"] = [f'{i}/{j}' for i, j in zip(final_df["chapter"], final_df["topic"])]
final_df["result_page"]=[int(i) for i in final_df["result_page"]]
final_df

In [ ]:
final_df["result_page"]=final_df["result_page"]+12
final_df["page_no"]=final_df["page_no"]+12
final_df["chapter_page"]=final_df["chapter_page"]+12

In [ ]:
print(final_df.dtypes)

In [ ]:
final_df.to_csv("final_df.csv",index=False)

In [ ]:
def compress_rows(input_csv, output_csv, group_by_column):
    # Read the input CSV into a DataFrame
    df = pd.read_csv(input_csv)

    # Group by the column with same elements and aggregate the other columns as lists
    grouped_df = df.groupby(group_by_column).agg(lambda x: list(x)).reset_index()

    # Write the compressed DataFrame to a new CSV
    grouped_df.to_csv(output_csv, index=False)

In [ ]:
# Example usage:
input_csv = 'final_df.csv'  # The input CSV file
output_csv = 'compressed_results.csv'  # The output CSV file
group_by_column = ['question',"generated_answer"]  # The column to group by (e.g., 'question')

compress_rows(input_csv, output_csv, group_by_column)

In [ ]:
comp_csv=pd.read_csv("compressed_results.csv")
# Rename the columns using the mapping

# Define the column renaming mapping
column_mapping = {
    'result_page': 'references',  # Rename 'result_page' to 'page_number'
    'result_text': 'context',  # Rename 'result_text' to 'page_text'
    'generated_answer': 'answer',  # Rename 'generated_answer' to 'answer'
}
comp_csv.rename(columns=column_mapping, inplace=True)

In [ ]:
cols_to_drop=["chapter_page","page_no","distance","chapter","topic"]

comp_csv.drop(columns=cols_to_drop,inplace=True)

In [ ]:
comp_csv.rename(columns={"references":"pages"},inplace=True)

In [ ]:
# Specify the columns you want to include in the key-value pair
selected_columns = ['sections', 'pages']

# Convert the values from selected columns into a dictionary
comp_csv['references'] = comp_csv[selected_columns].apply(lambda row: {col: row[col] for col in selected_columns}, axis=1)

In [ ]:
comp_csv.drop(columns=["pages","sections"],inplace=True)
comp_csv

In [ ]:
merged_df=df.merge(comp_csv,on="question", how="inner")
merged_df

In [ ]:
import ast

merged_df["references"]=[{"sections": ast.literal_eval(ref['sections']),"pages": ast.literal_eval(ref['pages'])} for ref in merged_df["references"]]

In [ ]:
merged_df.rename(columns= {'query_id': 'ID'}, inplace=True)

merged_df= merged_df[['ID', 'context', 'answer', 'references']]

merged_df

In [ ]:
# # Convert the list into the desired string format
# output_string = ["<" + "".join(item.strip("'") for item in input_list) + ">" for input_list in merged_df["context"]]
# # Replace the opening and closing brackets
# cleaned_text = [input_text.replace("<[", "<").replace("]>", ">") for input_text in output_string]

# merged_df["context"]=cleaned_text

merged_df.to_csv("submission.csv",index=False)

In [ ]:
merged_df["references"][0]